# Role Migration Pipeline

This notebook contains the complete pipeline for migrating from the ScheduleBot system to a role-based notification system.

## Pipeline Steps:
1. **Script 0**: Fetch all existing Discord roles
2. **Script 1**: Extract historical schedule bot events and current Discord events
3. **Script 2**: Map events to roles (using AI if needed)
4. **Script 3**: Generate user-to-role assignments from historical data
5. **Script 4**: Apply roles to users with rate limiting

## Setup and Configuration

In [8]:
# Install required packages
!pip install discord.py pandas mysql-connector-python python-dotenv aiohttp ratelimit openai langchain

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import os
import json
import asyncio
import pandas as pd
from datetime import datetime
import discord
from discord.ext import commands
import mysql.connector
from typing import List, Dict, Tuple
import aiohttp
from ratelimit import limits, sleep_and_retry
import csv

# Load environment variables
from dotenv import load_dotenv

# Try to load from the current directory first
env_path = os.path.join(os.getcwd(), '.env')
if os.path.exists(env_path):
    load_dotenv(env_path)
    print(f"Loaded .env from: {env_path}")
else:
    # Fall back to default behavior
    load_dotenv()
    print("Using default .env loading")

# Configuration
DISCORD_BOT_TOKEN = os.getenv('DISCORD_BOT_TOKEN')
MYSQL_SERVER = os.getenv('MYSQL_SERVER', 'scheduleBot-db')
MYSQL_DB = os.getenv('MYSQL_DB', 'ScheduleBot')
MYSQL_USER = os.getenv('MYSQL_USER', 'schedulebot')
MYSQL_PASSWORD = os.getenv('MYSQL_USER_PW', 'schedulebot')

# Debug: Print loaded values (without exposing sensitive data)
print("\nEnvironment variables loaded:")
print(f"DISCORD_BOT_TOKEN: {'✓ Set' if DISCORD_BOT_TOKEN else '✗ Not set'}")
print(f"MYSQL_SERVER: {MYSQL_SERVER}")
print(f"MYSQL_DB: {MYSQL_DB}")
print(f"MYSQL_USER: {MYSQL_USER}")
print(f"MYSQL_PASSWORD: {'✓ Set' if MYSQL_PASSWORD != 'schedulebot' else '✗ Using default'}")

# Output directories
OUTPUT_DIR = 'outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)

Loaded .env from: d:\bench\ScheduleBot\role_migration_system\.env

Environment variables loaded:
DISCORD_BOT_TOKEN: ✓ Set
MYSQL_SERVER: terraform-20250123164008602600000001.cqcqvqkwjtl5.us-east-1.rds.amazonaws.com
MYSQL_DB: schedulebot
MYSQL_USER: schedulebot
MYSQL_PASSWORD: ✓ Set


## Script 0: Fetch All Discord Roles

In [10]:
async def fetch_all_discord_roles():
    """
    Fetch all roles from the Discord server and save them to a CSV file.
    This will help us understand the naming patterns and filter relevant roles.
    """
    # Set up intents
    intents = discord.Intents.default()
    intents.guilds = True
    intents.members = True
    
    # Create bot instance
    bot = commands.Bot(command_prefix='!', intents=intents)
    
    all_roles = []
    
    @bot.event
    async def on_ready():
        print(f'Bot connected as {bot.user}')
        
        # Get all guilds (servers) the bot is in
        for guild in bot.guilds:
            print(f'\nFetching roles from: {guild.name} (ID: {guild.id})')
            
            # Get all roles in the guild
            for role in guild.roles:
                role_data = {
                    'guild_id': guild.id,
                    'guild_name': guild.name,
                    'role_id': role.id,
                    'role_name': role.name,
                    'role_color': str(role.color),
                    'role_position': role.position,
                    'is_bot_role': role.is_bot_managed(),
                    'is_default': role.is_default(),
                    'member_count': len(role.members),
                    'created_at': role.created_at.isoformat()
                }
                all_roles.append(role_data)
                
                # Print roles that might be class-related
                if 'class' in role.name.lower() or 'session' in role.name.lower():
                    print(f"  - {role.name} (Members: {len(role.members)})")
        
        # Save to CSV
        df = pd.DataFrame(all_roles)
        output_path = os.path.join(OUTPUT_DIR, 'all_discord_roles.csv')
        df.to_csv(output_path, index=False)
        print(f"\nSaved {len(all_roles)} roles to {output_path}")
        
        # Also save a filtered version with potential class roles
        class_keywords = ['class', 'session', 'workshop', 'practice', 'training', 'lesson']
        filtered_roles = df[
            df['role_name'].str.lower().str.contains('|'.join(class_keywords), na=False) &
            ~df['is_bot_role'] &
            ~df['is_default']
        ]
        
        filtered_path = os.path.join(OUTPUT_DIR, 'filtered_class_roles.csv')
        filtered_roles.to_csv(filtered_path, index=False)
        print(f"Saved {len(filtered_roles)} potential class roles to {filtered_path}")
        
        await bot.close()
    
    # Run the bot
    try:
        await bot.start(DISCORD_BOT_TOKEN)
    except Exception as e:
        print(f"Error: {e}")
        await bot.close()

# Run the script
# await fetch_all_discord_roles()

## Script 1: Extract Historical Events Data

In [11]:
def get_database_connection():
    """Create a connection to the MySQL database"""
    try:
        print(f"Attempting to connect to MySQL...")
        print(f"Host: {MYSQL_SERVER}")
        print(f"Database: {MYSQL_DB}")
        print(f"User: {MYSQL_USER}")
        
        connection = mysql.connector.connect(
            host=MYSQL_SERVER,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD,
            database=MYSQL_DB,
            connect_timeout=30  # 30 second timeout
        )
        print("✓ Successfully connected to database!")
        return connection
    except mysql.connector.Error as err:
        print(f"✗ MySQL Error: {err}")
        print(f"Error Code: {err.errno if hasattr(err, 'errno') else 'N/A'}")
        if err.errno == 2003:
            print("Can't connect to MySQL server - Check if:")
            print("1. The server address is correct")
            print("2. The database is publicly accessible")
            print("3. Security group allows connections from your IP")
            print("4. The server is running")
        elif err.errno == 1045:
            print("Access denied - Check username and password")
        return None
    except Exception as e:
        print(f"✗ General Error: {type(e).__name__}: {e}")
        return None

def extract_schedule_bot_events():
    """
    Extract all historical events from the ScheduleBot database
    """
    conn = get_database_connection()
    if not conn:
        return None
    
    try:
        # Query to get all schedule events with attendance data
        query = """
        SELECT 
            s.Id,
            s.EventTitle,
            s.EventDescription,
            s.StartTime,
            s.EndTime,
            s.ServerId,
            s.EventId,
            s.HostId,
            s.HostName,
            s.RoleId,
            s.IsActive,
            s.HasEnded,
            COUNT(DISTINCT a.UserId) as AttendeeCount,
            GROUP_CONCAT(DISTINCT a.UserId) as AttendeeIds,
            GROUP_CONCAT(DISTINCT a.Name) as AttendeeNames,
            GROUP_CONCAT(a.Status) as AttendeeStatuses
        FROM Schedules s
        LEFT JOIN Attenants a ON s.Id = a.ScheduleId
        GROUP BY s.Id
        ORDER BY s.StartTime DESC
        """
        
        df = pd.read_sql(query, conn)
        
        # Save the raw data
        output_path = os.path.join(OUTPUT_DIR, 'schedule_bot_events.csv')
        df.to_csv(output_path, index=False)
        print(f"Extracted {len(df)} schedule bot events to {output_path}")
        
        # Print summary statistics
        print(f"\nSummary:")
        print(f"Total events: {len(df)}")
        print(f"Active events: {len(df[df['IsActive'] == True])}")
        print(f"Events with attendees: {len(df[df['AttendeeCount'] > 0])}")
        print(f"\nMost common event titles:")
        print(df['EventTitle'].value_counts().head(10))
        
        return df
        
    except Exception as e:
        print(f"Error extracting schedule bot events: {e}")
        return None
    finally:
        conn.close()

# Run the extraction
# schedule_events_df = extract_schedule_bot_events()

async def fetch_discord_scheduled_events():
    """
    Fetch all scheduled events from Discord using the API
    """
    intents = discord.Intents.default()
    intents.guilds = True
    
    bot = commands.Bot(command_prefix='!', intents=intents)
    
    all_events = []
    
    @bot.event
    async def on_ready():
        print(f'Bot connected as {bot.user}')
        
        for guild in bot.guilds:
            print(f'\nFetching scheduled events from: {guild.name}')
            
            # Use direct API call to get scheduled events
            headers = {
                'Authorization': f'Bot {DISCORD_BOT_TOKEN}',
                'Content-Type': 'application/json'
            }
            
            async with aiohttp.ClientSession() as session:
                # Get all scheduled events
                url = f'https://discord.com/api/v10/guilds/{guild.id}/scheduled-events?with_user_count=true'
                async with session.get(url, headers=headers) as response:
                    if response.status == 200:
                        events = await response.json()
                        
                        for event in events:
                            event_data = {
                                'guild_id': guild.id,
                                'guild_name': guild.name,
                                'event_id': event['id'],
                                'event_name': event['name'],
                                'description': event.get('description', ''),
                                'start_time': event['scheduled_start_time'],
                                'end_time': event.get('scheduled_end_time'),
                                'status': event['status'],
                                'entity_type': event['entity_type'],
                                'user_count': event.get('user_count', 0),
                                'subscriber_ids': [],
                                'subscriber_count': 0
                            }
                            
                            # Get users who are interested in the event
                            # Note: This endpoint requires the event to be active/scheduled
                            if event['status'] in [1, 2]:  # SCHEDULED or ACTIVE
                                interested_url = f"https://discord.com/api/v10/guilds/{guild.id}/scheduled-events/{event['id']}/users?limit=1000&with_member=false"
                                
                                all_interested_users = []
                                after_user_id = None
                                
                                # Paginate through all interested users
                                while True:
                                    paginated_url = interested_url
                                    if after_user_id:
                                        paginated_url += f"&after={after_user_id}"
                                    
                                    async with session.get(paginated_url, headers=headers) as sub_response:
                                        if sub_response.status == 200:
                                            users_batch = await sub_response.json()
                                            if not users_batch:
                                                break
                                            
                                            all_interested_users.extend(users_batch)
                                            
                                            # If we got less than limit, we're done
                                            if len(users_batch) < 1000:
                                                break
                                            
                                            # Set up for next page
                                            after_user_id = users_batch[-1]['user']['id']
                                        else:
                                            print(f"    Failed to get interested users for {event['name']}: {sub_response.status}")
                                            break
                                
                                event_data['subscriber_ids'] = [user['user']['id'] for user in all_interested_users]
                                event_data['subscriber_count'] = len(all_interested_users)
                            
                            all_events.append(event_data)
                            print(f"  - {event['name']} (Interested: {event_data['subscriber_count']}, Status: {event['status']})")
        
        # Save to CSV
        df = pd.DataFrame(all_events)
        output_path = os.path.join(OUTPUT_DIR, 'discord_scheduled_events.csv')
        df.to_csv(output_path, index=False)
        print(f"\nSaved {len(all_events)} Discord scheduled events to {output_path}")
        
        await bot.close()
    
    try:
        await bot.start(DISCORD_BOT_TOKEN)
    except Exception as e:
        print(f"Error: {e}")
        await bot.close()

# Run the script
# await fetch_discord_scheduled_events()

In [12]:
# Debug: Let's check what we got from Discord events
discord_events_df = pd.read_csv(os.path.join(OUTPUT_DIR, 'discord_scheduled_events.csv'))
print(f"Total Discord events: {len(discord_events_df)}")
print(f"\nEvents with subscribers:")
events_with_subs = discord_events_df[discord_events_df['subscriber_count'] > 0]
print(events_with_subs[['event_name', 'subscriber_count', 'start_time']].to_string())

print(f"\nEvents without subscribers:")
events_without_subs = discord_events_df[discord_events_df['subscriber_count'] == 0]
print(f"Count: {len(events_without_subs)}")

# Check date ranges
discord_events_df['start_time'] = pd.to_datetime(discord_events_df['start_time'])
print(f"\nDate range of events: {discord_events_df['start_time'].min()} to {discord_events_df['start_time'].max()}")

Total Discord events: 26

Events with subscribers:
                                event_name  subscriber_count                 start_time
0                      Monday Night Movies               7.0  2025-08-05T01:30:00+00:00
1    Breakin' Dance Class with Elliesaurus              33.0  2025-08-04T21:00:00+00:00
2   Creative Coding Club with Joyful Decay               7.0  2025-08-07T21:00:00+00:00
3      VKET - Yoga / Stretching - BASICBIT              25.0  2025-08-07T23:00:00+00:00
4                    Vocal Class - FoxyFoo              30.0  2025-08-05T20:00:00+00:00
16            Yoga / Stretching - BASICBIT              27.0  2025-12-26T01:00:00+00:00
25                    Unity/Blender Class!               1.0  2025-08-06T01:00:00+00:00

Events without subscribers:
Count: 0

Date range of events: 2025-08-03 01:30:00+00:00 to 2025-12-26 01:00:00+00:00


In [13]:
async def fetch_discord_scheduled_events():
    """
    Fetch all scheduled events from Discord using the API
    """
    intents = discord.Intents.default()
    intents.guilds = True
    
    bot = commands.Bot(command_prefix='!', intents=intents)
    
    all_events = []
    
    @bot.event
    async def on_ready():
        print(f'Bot connected as {bot.user}')
        
        for guild in bot.guilds:
            print(f'\nFetching scheduled events from: {guild.name}')
            
            # Use direct API call to get scheduled events
            headers = {
                'Authorization': f'Bot {DISCORD_BOT_TOKEN}',
                'Content-Type': 'application/json'
            }
            
            async with aiohttp.ClientSession() as session:
                # Get all scheduled events
                url = f'https://discord.com/api/v10/guilds/{guild.id}/scheduled-events'
                async with session.get(url, headers=headers) as response:
                    if response.status == 200:
                        events = await response.json()
                        
                        for event in events:
                            event_data = {
                                'guild_id': guild.id,
                                'guild_name': guild.name,
                                'event_id': event['id'],
                                'event_name': event['name'],
                                'description': event.get('description', ''),
                                'start_time': event['scheduled_start_time'],
                                'end_time': event.get('scheduled_end_time'),
                                'status': event['status'],
                                'entity_type': event['entity_type'],
                                'user_count': event.get('user_count', 0)
                            }
                            all_events.append(event_data)
                            
                            # Get event subscribers if available
                            subscribers_url = f"{url}/{event['id']}/users?limit=100"
                            async with session.get(subscribers_url, headers=headers) as sub_response:
                                if sub_response.status == 200:
                                    subscribers = await sub_response.json()
                                    event_data['subscriber_ids'] = [sub['user']['id'] for sub in subscribers]
                                    event_data['subscriber_count'] = len(subscribers)
                            
                            print(f"  - {event['name']} (Subscribers: {event_data.get('subscriber_count', 0)})")
        
        # Save to CSV
        df = pd.DataFrame(all_events)
        output_path = os.path.join(OUTPUT_DIR, 'discord_scheduled_events.csv')
        df.to_csv(output_path, index=False)
        print(f"\nSaved {len(all_events)} Discord scheduled events to {output_path}")
        
        await bot.close()
    
    try:
        await bot.start(DISCORD_BOT_TOKEN)
    except Exception as e:
        print(f"Error: {e}")
        await bot.close()

# Run the script
# await fetch_discord_scheduled_events()

## Run Script 0 - Fetch Discord Roles

Let's run this first to see what roles are available:

In [ ]:
async def fetch_all_guild_members():
    """
    Fetch all members from The Faceless Discord server with their full details.
    """
    # Set up intents - we need members intent
    intents = discord.Intents.default()
    intents.guilds = True
    intents.members = True  # Required to fetch all members
    
    # Create bot instance
    bot = commands.Bot(command_prefix='!', intents=intents)
    
    all_members = []
    
    @bot.event
    async def on_ready():
        print(f'Bot connected as {bot.user}')
        
        # Target The Faceless server
        FACELESS_GUILD_ID = 480695542155051010
        guild = bot.get_guild(FACELESS_GUILD_ID)
        
        if not guild:
            print(f"ERROR: Could not find The Faceless guild (ID: {FACELESS_GUILD_ID})")
            await bot.close()
            return
        
        print(f"\\nFetching members from: {guild.name} (ID: {guild.id})")
        print(f"Total members: {guild.member_count}")
        
        # Fetch all members - this might take a while for large servers
        print("Fetching all members... (this may take a moment)")
        
        # Use chunk_guilds to ensure we have all members
        await bot.chunk_guild(guild)
        
        members_processed = 0
        
        for member in guild.members:
            member_data = {
                'user_id': member.id,
                'username': member.name,  # Discord username
                'display_name': member.display_name,  # Server nickname or username
                'nick': member.nick,  # Server nickname (None if not set)
                'discriminator': member.discriminator,  # The 4-digit tag (legacy)
                'global_name': member.global_name if hasattr(member, 'global_name') else None,  # New global display name
                'bot': member.bot,
                'joined_at': member.joined_at.isoformat() if member.joined_at else None,
                'created_at': member.created_at.isoformat(),
                'roles': [role.name for role in member.roles if role.name != '@everyone'],
                'role_ids': [str(role.id) for role in member.roles if role.name != '@everyone'],
                'top_role': member.top_role.name if member.top_role.name != '@everyone' else None,
                'status': str(member.status) if hasattr(member, 'status') else 'unknown',
                'premium_since': member.premium_since.isoformat() if member.premium_since else None,
            }
            
            all_members.append(member_data)
            members_processed += 1
            
            # Progress update
            if members_processed % 100 == 0:
                print(f"  Processed {members_processed}/{guild.member_count} members...")
        
        print(f"\\nSuccessfully fetched {len(all_members)} members")
        
        # Convert to DataFrame
        df = pd.DataFrame(all_members)
        
        # Save full member list
        output_path = os.path.join(OUTPUT_DIR, 'faceless_all_members.csv')
        df.to_csv(output_path, index=False)
        print(f"Saved full member list to: {output_path}")
        
        # Also save a simplified version for username mapping
        simple_df = df[['user_id', 'username', 'display_name', 'nick', 'global_name', 'bot']].copy()
        simple_output_path = os.path.join(OUTPUT_DIR, 'faceless_members_simple.csv')
        simple_df.to_csv(simple_output_path, index=False)
        print(f"Saved simplified member list to: {simple_output_path}")
        
        # Create a username lookup table (all possible names a user might be known by)
        lookup_data = []
        for _, member in df.iterrows():
            user_id = member['user_id']
            
            # Add all possible name variations
            names_to_add = set()
            
            if pd.notna(member['username']):
                names_to_add.add(member['username'])
            if pd.notna(member['display_name']):
                names_to_add.add(member['display_name'])
            if pd.notna(member['nick']):
                names_to_add.add(member['nick'])
            if pd.notna(member['global_name']):
                names_to_add.add(member['global_name'])
            
            for name in names_to_add:
                lookup_data.append({
                    'lookup_name': name.lower(),  # Store lowercase for case-insensitive matching
                    'original_name': name,
                    'user_id': user_id,
                    'username': member['username'],
                    'is_bot': member['bot']
                })
        
        lookup_df = pd.DataFrame(lookup_data)
        lookup_output_path = os.path.join(OUTPUT_DIR, 'faceless_username_lookup.csv')
        lookup_df.to_csv(lookup_output_path, index=False)
        print(f"Saved username lookup table to: {lookup_output_path}")
        
        # Print summary statistics
        print(f"\\nSummary:")
        print(f"  Total members: {len(df)}")
        print(f"  Human members: {len(df[~df['bot']])}")
        print(f"  Bot members: {len(df[df['bot']])}")
        print(f"  Members with nicknames: {len(df[df['nick'].notna()])}")
        print(f"  Unique usernames in lookup: {len(lookup_df['lookup_name'].unique())}")
        
        await bot.close()
    
    try:
        await bot.start(DISCORD_BOT_TOKEN)
    except Exception as e:
        print(f"Error: {e}")
        await bot.close()

# Run this to fetch all Discord members
await fetch_all_guild_members()

Bot connected as Meeting Notes#2283
\nFetching members from: The Faceless (ID: 480695542155051010)
Total members: 1318
Fetching all members... (this may take a moment)


## Fetch All Discord Members

Before we can map manual usernames to user IDs, we need to fetch all members from The Faceless Discord server:

In [ ]:
# Run Script 0 to fetch all Discord roles
# This will help us understand the naming patterns
await fetch_all_discord_roles()

Bot connected as Meeting Notes#2283

Fetching roles from: DDM (ID: 266269732565549061)

Fetching roles from: The Faceless (ID: 480695542155051010)
  - Art Classes (Members: 0)
  - ASL Class (Members: 0)
  - DJ Class (Members: 20)
  - Vocal Class (Members: 0)

Fetching roles from: Rathos' Picture Den (No Deleto) (ID: 677740206337818654)

Fetching roles from: BASIC's Creations (ID: 1249723747896918109)
  - Class Announcements (Members: 5)

Saved 334 roles to outputs\all_discord_roles.csv
Saved 5 potential class roles to outputs\filtered_class_roles.csv


## Analyze Roles and Filter for Target Servers

In [ ]:
# Load and analyze the roles data
roles_df = pd.read_csv(os.path.join(OUTPUT_DIR, 'all_discord_roles.csv'))

# Filter for our target servers
PRODUCTION_SERVER_ID = 480695542155051010  # The Faceless
TEST_SERVER_ID = 1249723747896918109  # BASIC's Creations

# Get dance/class related roles from production server
production_roles = roles_df[roles_df['guild_id'] == PRODUCTION_SERVER_ID]

# Define keywords for dance/class roles (expanded list)
dance_keywords = [
    'class', 'dance', 'breakin', 'poppin', 'lockin', 'house', 
    'hip hop', 'yoga', 'vocal', 'dj', 'freestyle', 'gogo',
    'robot', 'animation', 'vogue', 'waack', 'choreo', 'movement',
    'art', 'asl', 'podcast',
    # New keywords added
    'photography', 'scare', 'sociology', 'music', 'loft', 'vibe',
    'bachata', 'waltz', 'esoteric', 'blender', 'unity', 'coding',
    'dancer'
]

# Filter for relevant roles (case insensitive)
relevant_roles = production_roles[
    production_roles['role_name'].str.lower().str.contains('|'.join(dance_keywords), na=False) &
    ~production_roles['is_bot_role'] &
    ~production_roles['is_default']
    # Removed the exclusion of "Dancer" role
].sort_values('role_name')

print(f"Found {len(relevant_roles)} relevant roles in production server:")
print("\\nRole Name | Current Members | Role ID")
print("-" * 50)
for _, role in relevant_roles.iterrows():
    print(f"{role['role_name']:<25} | {role['member_count']:>5} | {role['role_id']}")

# Save filtered roles
relevant_roles_path = os.path.join(OUTPUT_DIR, 'relevant_class_roles.csv')
relevant_roles.to_csv(relevant_roles_path, index=False)
print(f"\\nSaved relevant roles to {relevant_roles_path}")

Found 33 relevant roles in production server:
\nRole Name | Current Members | Role ID
--------------------------------------------------
ASL Class                 |     0 | 1392210986387116173
Animation Dance           |     0 | 1392209677282185446
Art Classes               |     0 | 1392212893100933352
Bachata                   |     0 | 1392210116924674212
Beyond Dance              |     0 | 1392213330457923594
Blender/Unity             |     0 | 1392210837330067576
Breakin'                  |     0 | 1392210704140079285
Coding                    |     0 | 1392210886487445655
Community Manager         |     1 | 1210262400985727066
DJ                        |   280 | 1163047310889062420
DJ Class                  |    20 | 1392210777888526366
Dancer                    |   756 | 726369852616474665
Esoterics                 |     0 | 1392210454822260737
Flow Arts                 |     0 | 1392213369284591787
Freestyle Dance           |     0 | 1392213565519167548
GOGO Dance              

## Script 2: Smart Event-to-Role Mapping

Based on the role names, we might be able to do simple pattern matching instead of using AI.

In [ ]:
def generate_user_role_assignments_with_manual(schedule_events_df, discord_events_df, mappings_df):
    """
    Generate user-to-role assignments based on historical attendance data,
    Discord event subscribers, and manually collected Discord interests.
    """
    # Load the reviewed mappings (user should have edited the CSV if needed)
    mappings_path = os.path.join(OUTPUT_DIR, 'event_role_mappings_reviewed.csv')
    if os.path.exists(mappings_path):
        print("Loading reviewed mappings...")
        mappings_df = pd.read_csv(mappings_path)
    else:
        print("Using automatic mappings (no reviewed file found)...")
    
    # Load manual Discord interests with user IDs
    manual_interests_path = os.path.join(OUTPUT_DIR, 'manual_discord_interests_with_ids.csv')
    manual_interests_df = None
    if os.path.exists(manual_interests_path):
        print("Loading manual Discord interests...")
        manual_interests_df = pd.read_csv(manual_interests_path)
        print(f"  Loaded {len(manual_interests_df)} manual interest records")
    
    # Dictionary to store user-role assignments
    # Format: {role_id: {user_id: {'username': name, 'sources': [list of events]}}}
    user_role_assignments = {}
    
    print("\\nProcessing Schedule Bot attendance data...")
    print("=" * 80)
    
    # Process schedule bot events
    if schedule_events_df is not None:
        for _, event in schedule_events_df.iterrows():
            event_title = event['EventTitle']
            
            # Find the role mapping for this event
            mapping = mappings_df[
                (mappings_df['event_source'] == 'schedule_bot') & 
                (mappings_df['event_title'] == event_title)
            ]
            
            if len(mapping) == 0 or pd.isna(mapping.iloc[0]['matched_role_id']):
                continue
            
            role_id = int(mapping.iloc[0]['matched_role_id'])
            role_name = mapping.iloc[0]['matched_role_name']
            
            # Initialize role dict if needed
            if role_id not in user_role_assignments:
                user_role_assignments[role_id] = {}
            
            # Process attendees
            if pd.notna(event['AttendeeIds']) and event['AttendeeCount'] > 0:
                attendee_ids = str(event['AttendeeIds']).split(',')
                attendee_names = str(event['AttendeeNames']).split(',')
                
                for i, user_id in enumerate(attendee_ids):
                    try:
                        user_id = int(user_id.strip())
                        username = attendee_names[i].strip() if i < len(attendee_names) else 'Unknown'
                        
                        if user_id not in user_role_assignments[role_id]:
                            user_role_assignments[role_id][user_id] = {
                                'username': username,
                                'sources': []
                            }
                        
                        user_role_assignments[role_id][user_id]['sources'].append({
                            'type': 'schedule_bot',
                            'event': event_title,
                            'date': str(event['StartTime'])
                        })
                    except (ValueError, IndexError):
                        continue
        
        print(f"Processed {len(schedule_events_df)} schedule bot events")
    
    print("\\nProcessing Discord scheduled event subscribers...")
    print("=" * 80)
    
    # Process Discord scheduled events
    if discord_events_df is not None and len(discord_events_df) > 0:
        for _, event in discord_events_df.iterrows():
            event_name = event['event_name']
            
            # Find the role mapping for this event
            mapping = mappings_df[
                (mappings_df['event_source'] == 'discord_scheduled') & 
                (mappings_df['event_title'] == event_name)
            ]
            
            if len(mapping) == 0 or pd.isna(mapping.iloc[0]['matched_role_id']):
                continue
            
            role_id = int(mapping.iloc[0]['matched_role_id'])
            role_name = mapping.iloc[0]['matched_role_name']
            
            # Initialize role dict if needed
            if role_id not in user_role_assignments:
                user_role_assignments[role_id] = {}
            
            # Process subscribers
            if pd.notna(event.get('subscriber_ids')):
                subscriber_ids = eval(event['subscriber_ids']) if isinstance(event['subscriber_ids'], str) else event['subscriber_ids']
                
                for user_id in subscriber_ids:
                    try:
                        user_id = int(user_id)
                        
                        if user_id not in user_role_assignments[role_id]:
                            user_role_assignments[role_id][user_id] = {
                                'username': 'Discord Event Subscriber',
                                'sources': []
                            }
                        
                        user_role_assignments[role_id][user_id]['sources'].append({
                            'type': 'discord_scheduled',
                            'event': event_name,
                            'date': str(event['start_time'])
                        })
                    except ValueError:
                        continue
        
        print(f"Processed {len(discord_events_df)} Discord scheduled events")
    
    print("\\nProcessing Manual Discord Interests...")
    print("=" * 80)
    
    # Process manual Discord interests
    if manual_interests_df is not None:
        manual_users_added = 0
        
        for _, interest in manual_interests_df.iterrows():
            if pd.notna(interest['user_id']) and pd.notna(interest['role_id']):
                try:
                    user_id = int(interest['user_id'])
                    role_id = int(interest['role_id'])
                    role_name = interest['role_name']
                    
                    # Initialize role dict if needed
                    if role_id not in user_role_assignments:
                        user_role_assignments[role_id] = {}
                    
                    # Add user if not already there
                    if user_id not in user_role_assignments[role_id]:
                        user_role_assignments[role_id][user_id] = {
                            'username': interest['username'],
                            'sources': []
                        }
                        manual_users_added += 1
                    
                    # Add the manual interest as a source
                    user_role_assignments[role_id][user_id]['sources'].append({
                        'type': 'manual_discord_interest',
                        'event': interest['event_name'],
                        'date': 'manual_collection'
                    })
                except (ValueError, TypeError):
                    continue
        
        print(f"Added {manual_users_added} users from manual Discord interests")
    
    # Convert to flat list for CSV output
    assignments_list = []
    for role_id, users in user_role_assignments.items():
        # Get role name from mappings
        role_info = mappings_df[mappings_df['matched_role_id'] == role_id]
        if len(role_info) > 0:
            role_name = role_info.iloc[0]['matched_role_name']
        else:
            role_name = f"Unknown Role ({role_id})"
        
        for user_id, user_data in users.items():
            assignments_list.append({
                'role_id': role_id,
                'role_name': role_name,
                'user_id': user_id,
                'username': user_data['username'],
                'event_count': len(user_data['sources']),
                'first_event': min(s['date'] for s in user_data['sources']),
                'last_event': max(s['date'] for s in user_data['sources']),
                'source_types': ','.join(sorted(set(s['type'] for s in user_data['sources'])))
            })
    
    # Create DataFrame and save
    assignments_df = pd.DataFrame(assignments_list)
    assignments_df = assignments_df.sort_values(['role_name', 'event_count'], ascending=[True, False])
    
    # Save detailed assignments
    assignments_path = os.path.join(OUTPUT_DIR, 'user_role_assignments_combined_for_review.csv')
    assignments_df.to_csv(assignments_path, index=False)
    
    # Create summary statistics
    print("\\nAssignment Summary:")
    print("=" * 80)
    print(f"Total unique users to be assigned roles: {len(assignments_df['user_id'].unique())}")
    print(f"Total role assignments to be made: {len(assignments_df)}")
    
    # Show breakdown by source type
    print("\\nUsers by source type:")
    for source_type in ['schedule_bot', 'discord_scheduled', 'manual_discord_interest']:
        users_with_source = assignments_df[assignments_df['source_types'].str.contains(source_type)]
        print(f"  - {source_type}: {len(users_with_source['user_id'].unique())} unique users")
    
    print("\\nAssignments per role:")
    
    role_summary = assignments_df.groupby('role_name').agg({
        'user_id': 'count',
        'event_count': 'sum'
    }).rename(columns={'user_id': 'users', 'event_count': 'total_events'})
    
    print(role_summary.to_string())
    
    print(f"\\nDetailed assignments saved to: {assignments_path}")
    print("\\nPlease review the assignments file before proceeding to the final step.")
    
    return assignments_df

# This will be run after mappings are reviewed
# assignments_df = generate_user_role_assignments_with_manual(schedule_events_df, discord_events_df, mappings_df)

## Script 3: Generate User-to-Role Assignments

In [ ]:
def generate_user_role_assignments(schedule_events_df, discord_events_df, mappings_df):
    """
    Generate user-to-role assignments based on historical attendance data
    and Discord event subscribers.
    """
    # Load the reviewed mappings (user should have edited the CSV if needed)
    mappings_path = os.path.join(OUTPUT_DIR, 'event_role_mappings_reviewed.csv')
    if os.path.exists(mappings_path):
        print("Loading reviewed mappings...")
        mappings_df = pd.read_csv(mappings_path)
    else:
        print("Using automatic mappings (no reviewed file found)...")
    
    # Dictionary to store user-role assignments
    # Format: {role_id: {user_id: {'username': name, 'sources': [list of events]}}}
    user_role_assignments = {}
    
    print("Processing Schedule Bot attendance data...")
    print("=" * 80)
    
    # Process schedule bot events
    if schedule_events_df is not None:
        for _, event in schedule_events_df.iterrows():
            event_title = event['EventTitle']
            
            # Find the role mapping for this event
            mapping = mappings_df[
                (mappings_df['event_source'] == 'schedule_bot') & 
                (mappings_df['event_title'] == event_title)
            ]
            
            if len(mapping) == 0 or pd.isna(mapping.iloc[0]['matched_role_id']):
                continue
            
            role_id = int(mapping.iloc[0]['matched_role_id'])
            role_name = mapping.iloc[0]['matched_role_name']
            
            # Initialize role dict if needed
            if role_id not in user_role_assignments:
                user_role_assignments[role_id] = {}
            
            # Process attendees
            if pd.notna(event['AttendeeIds']) and event['AttendeeCount'] > 0:
                attendee_ids = str(event['AttendeeIds']).split(',')
                attendee_names = str(event['AttendeeNames']).split(',')
                
                for i, user_id in enumerate(attendee_ids):
                    try:
                        user_id = int(user_id.strip())
                        username = attendee_names[i].strip() if i < len(attendee_names) else 'Unknown'
                        
                        if user_id not in user_role_assignments[role_id]:
                            user_role_assignments[role_id][user_id] = {
                                'username': username,
                                'sources': []
                            }
                        
                        user_role_assignments[role_id][user_id]['sources'].append({
                            'type': 'schedule_bot',
                            'event': event_title,
                            'date': str(event['StartTime'])  # Convert to string
                        })
                    except (ValueError, IndexError):
                        continue
        
        print(f"Processed {len(schedule_events_df)} schedule bot events")
    
    print("\\nProcessing Discord scheduled event subscribers...")
    print("=" * 80)
    
    # Process Discord scheduled events
    if discord_events_df is not None and len(discord_events_df) > 0:
        for _, event in discord_events_df.iterrows():
            event_name = event['event_name']
            
            # Find the role mapping for this event
            mapping = mappings_df[
                (mappings_df['event_source'] == 'discord_scheduled') & 
                (mappings_df['event_title'] == event_name)
            ]
            
            if len(mapping) == 0 or pd.isna(mapping.iloc[0]['matched_role_id']):
                continue
            
            role_id = int(mapping.iloc[0]['matched_role_id'])
            role_name = mapping.iloc[0]['matched_role_name']
            
            # Initialize role dict if needed
            if role_id not in user_role_assignments:
                user_role_assignments[role_id] = {}
            
            # Process subscribers
            if pd.notna(event.get('subscriber_ids')):
                subscriber_ids = eval(event['subscriber_ids']) if isinstance(event['subscriber_ids'], str) else event['subscriber_ids']
                
                for user_id in subscriber_ids:
                    try:
                        user_id = int(user_id)
                        
                        if user_id not in user_role_assignments[role_id]:
                            user_role_assignments[role_id][user_id] = {
                                'username': 'Discord Event Subscriber',
                                'sources': []
                            }
                        
                        user_role_assignments[role_id][user_id]['sources'].append({
                            'type': 'discord_scheduled',
                            'event': event_name,
                            'date': str(event['start_time'])  # Convert to string
                        })
                    except ValueError:
                        continue
        
        print(f"Processed {len(discord_events_df)} Discord scheduled events")
    
    # Convert to flat list for CSV output
    assignments_list = []
    for role_id, users in user_role_assignments.items():
        # Get role name from mappings
        role_info = mappings_df[mappings_df['matched_role_id'] == role_id].iloc[0]
        role_name = role_info['matched_role_name']
        
        for user_id, user_data in users.items():
            assignments_list.append({
                'role_id': role_id,
                'role_name': role_name,
                'user_id': user_id,
                'username': user_data['username'],
                'event_count': len(user_data['sources']),
                'first_event': min(s['date'] for s in user_data['sources']),
                'last_event': max(s['date'] for s in user_data['sources']),
                'source_types': ','.join(set(s['type'] for s in user_data['sources']))
            })
    
    # Create DataFrame and save
    assignments_df = pd.DataFrame(assignments_list)
    assignments_df = assignments_df.sort_values(['role_name', 'event_count'], ascending=[True, False])
    
    # Save detailed assignments
    assignments_path = os.path.join(OUTPUT_DIR, 'user_role_assignments_for_review.csv')
    assignments_df.to_csv(assignments_path, index=False)
    
    # Create summary statistics
    print("\\nAssignment Summary:")
    print("=" * 80)
    print(f"Total unique users to be assigned roles: {len(assignments_df['user_id'].unique())}")
    print(f"Total role assignments to be made: {len(assignments_df)}")
    print("\\nAssignments per role:")
    
    role_summary = assignments_df.groupby('role_name').agg({
        'user_id': 'count',
        'event_count': 'sum'
    }).rename(columns={'user_id': 'users', 'event_count': 'total_events'})
    
    print(role_summary.to_string())
    
    print(f"\\nDetailed assignments saved to: {assignments_path}")
    print("\\nPlease review the assignments file before proceeding to the final step.")
    
    return assignments_df

# This will be run after mappings are reviewed
# assignments_df = generate_user_role_assignments(schedule_events_df, discord_events_df, mappings_df)

## Script 4: Apply Roles to Users (With Verification)

In [ ]:
@sleep_and_retry
@limits(calls=20, period=1)  # 20 requests per second
async def add_role_to_user(guild, user_id, role_id, session, headers):
    """Rate-limited function to add a role to a user"""
    url = f"https://discord.com/api/v10/guilds/{guild.id}/members/{user_id}/roles/{role_id}"
    async with session.put(url, headers=headers) as response:
        return response.status, user_id, role_id

async def apply_roles_to_users(assignments_df, target_guild_id, dry_run=True):
    """
    Apply roles to users based on the reviewed assignments.
    
    Args:
        assignments_df: DataFrame with user role assignments
        target_guild_id: The guild ID to apply roles to
        dry_run: If True, only simulate the actions without applying
    """
    # Load reviewed assignments if available
    reviewed_path = os.path.join(OUTPUT_DIR, 'user_role_assignments_reviewed.csv')
    if os.path.exists(reviewed_path):
        print("Loading reviewed assignments...")
        assignments_df = pd.read_csv(reviewed_path)
    else:
        print("Using automatic assignments (no reviewed file found)...")
    
    intents = discord.Intents.default()
    intents.guilds = True
    intents.members = True
    
    bot = commands.Bot(command_prefix='!', intents=intents)
    
    @bot.event
    async def on_ready():
        print(f'Bot connected as {bot.user}')
        
        # Get the target guild
        guild = bot.get_guild(target_guild_id)
        if not guild:
            print(f"ERROR: Could not find guild with ID {target_guild_id}")
            await bot.close()
            return
        
        print(f"\\nTarget Guild: {guild.name} (ID: {guild.id})")
        print(f"Mode: {'DRY RUN' if dry_run else 'LIVE - APPLYING ROLES'}")
        print("=" * 80)
        
        # Group assignments by role for efficiency
        role_groups = assignments_df.groupby(['role_id', 'role_name'])
        
        total_assignments = len(assignments_df)
        successful = 0
        failed = 0
        already_has_role = 0
        user_not_found = 0
        
        # Prepare for API calls
        headers = {
            'Authorization': f'Bot {DISCORD_BOT_TOKEN}',
            'Content-Type': 'application/json'
        }
        
        if not dry_run:
            # Final confirmation
            print(f"\\nABOUT TO APPLY {total_assignments} ROLE ASSIGNMENTS!")
            print("This action cannot be easily undone.")
            confirmation = input("Type 'YES' to proceed, anything else to cancel: ")
            
            if confirmation != 'YES':
                print("Operation cancelled.")
                await bot.close()
                return
        
        print("\\nProcessing role assignments...")
        
        async with aiohttp.ClientSession() as session:
            for (role_id, role_name), group in role_groups:
                role_id = int(role_id)
                role = guild.get_role(role_id)
                
                if not role:
                    print(f"\\nWARNING: Role '{role_name}' (ID: {role_id}) not found in guild!")
                    failed += len(group)
                    continue
                
                print(f"\\nProcessing role: {role_name} ({len(group)} users)")
                
                for _, assignment in group.iterrows():
                    user_id = int(assignment['user_id'])
                    username = assignment['username']
                    
                    try:
                        # Check if member exists in guild
                        member = guild.get_member(user_id)
                        
                        if not member:
                            # Try to fetch member
                            try:
                                member = await guild.fetch_member(user_id)
                            except discord.NotFound:
                                print(f"  ✗ User {username} ({user_id}) not found in guild")
                                user_not_found += 1
                                continue
                        
                        # Check if member already has the role
                        if role in member.roles:
                            print(f"  ⚠ {member.name} already has role {role_name}")
                            already_has_role += 1
                            continue
                        
                        if dry_run:
                            print(f"  [DRY RUN] Would add {role_name} to {member.name} ({member.id})")
                            successful += 1
                        else:
                            # Apply the role
                            status, _, _ = await add_role_to_user(guild, user_id, role_id, session, headers)
                            
                            if status == 204:  # Success
                                print(f"  ✓ Added {role_name} to {member.name} ({member.id})")
                                successful += 1
                            else:
                                print(f"  ✗ Failed to add {role_name} to {member.name} - Status: {status}")
                                failed += 1
                    
                    except Exception as e:
                        print(f"  ✗ Error processing user {username} ({user_id}): {str(e)}")
                        failed += 1
        
        # Final summary
        print("\\n" + "=" * 80)
        print("FINAL SUMMARY:")
        print(f"Total assignments processed: {total_assignments}")
        print(f"Successful: {successful}")
        print(f"Already had role: {already_has_role}")
        print(f"User not found: {user_not_found}")
        print(f"Failed: {failed}")
        
        # Save summary report
        summary = {
            'timestamp': datetime.now().isoformat(),
            'guild_id': target_guild_id,
            'guild_name': guild.name,
            'dry_run': dry_run,
            'total_assignments': total_assignments,
            'successful': successful,
            'already_has_role': already_has_role,
            'user_not_found': user_not_found,
            'failed': failed
        }
        
        summary_path = os.path.join(OUTPUT_DIR, f'role_application_summary_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json')
        with open(summary_path, 'w') as f:
            json.dump(summary, f, indent=2)
        
        print(f"\\nSummary saved to: {summary_path}")
        
        await bot.close()
    
    try:
        await bot.start(DISCORD_BOT_TOKEN)
    except Exception as e:
        print(f"Error: {e}")
        await bot.close()

# Run in dry-run mode first
# await apply_roles_to_users(assignments_df, TEST_SERVER_ID, dry_run=True)

# Then run for real after verification
# await apply_roles_to_users(assignments_df, PRODUCTION_SERVER_ID, dry_run=False)

## Pipeline Execution Steps

Run these cells in order:

In [ ]:
# Step 1: Re-run the cell above (cell-12) first to update the role filtering!
# Then run this cell to load the updated relevant roles
relevant_roles = pd.read_csv(os.path.join(OUTPUT_DIR, 'relevant_class_roles.csv'))
print(f"Loaded {len(relevant_roles)} relevant roles")

Loaded 33 relevant roles


In [ ]:
# Step 2: Extract historical schedule bot events
schedule_events_df = extract_schedule_bot_events()

Attempting to connect to MySQL...
Host: terraform-20250123164008602600000001.cqcqvqkwjtl5.us-east-1.rds.amazonaws.com
Database: schedulebot
User: schedulebot
✓ Successfully connected to database!


C:\Users\steve\AppData\Local\Temp\ipykernel_39192\2367585054.py:68: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Extracted 539 schedule bot events to outputs\schedule_bot_events.csv

Summary:
Total events: 539
Active events: 1
Events with attendees: 457

Most common event titles:
EventTitle
Sign Sessions - ASL club - Xanori         46
Community Night                           31
Breakin' Class with Ellie                 27
Faceless Meditation                       26
Wotagei with Ruriki                       25
Movie Night Mondays                       23
Creative Coding Club with Joyful Decay    22
Esoterics Class w/ FoxyFoo                19
Guide to Go-Go dancing with Gaster        16
3D Modeling with Lowdisan                 16
Name: count, dtype: int64


In [ ]:
# Step 5: Generate user-to-role assignments INCLUDING manual Discord interests
# This enhanced version includes the manually collected Discord event interests
assignments_df = generate_user_role_assignments_with_manual(schedule_events_df, discord_events_df, mappings_df)

# IMPORTANT: Review outputs/user_role_assignments_combined_for_review.csv and save your corrections
# as outputs/user_role_assignments_combined_reviewed.csv before proceeding!

In [86]:
# Step 4: Create event-to-role mappings
mappings_df = create_event_to_role_mapping(schedule_events_df, discord_events_df, relevant_roles)

# IMPORTANT: Review outputs/event_role_mappings_for_review.csv and save your corrections 
# as outputs/event_role_mappings_reviewed.csv before proceeding!

Analyzing Schedule Bot Events...
✓ 'Community Night' → 'Community Manager' (Score: 0.75)
✗ 'Faceless Meditation' - No match found
✗ 'Sign Sessions - ASL club - Xanori' - No match found
✓ 'Intro to DJing with Embex' → 'DJ' (Score: 1.00)
✓ 'Vibe Session - Lounge and Cypher' → 'VIBE' (Score: 1.00)
✗ 'Karokee Night' - No match found
✗ 'Guide to Go-Go dancing with Gaster' - No match found
✓ 'Yoga and Stretching with BASICBIT' → 'YOGA' (Score: 1.00)
✓ 'Creative Coding Club with Joyful Decay' → 'Coding' (Score: 1.00)
✓ 'ASL class with Wardragon' → 'ASL Class' (Score: 1.00)
✗ 'Producers Chat & Jam Session' - No match found
✗ 'Wotagei with Ruriki' - No match found
✗ '3D Modeling with Lowdisan' - No match found
✓ 'BEYOND DANCE w/ NJNA' → 'Beyond Dance' (Score: 1.00)
✓ 'Vocal classes with FoxyFoo' → 'Vocal Class' (Score: 1.00)
✗ 'Movie Night Mondays' - No match found
✓ 'Breakin' Class with Ellie' → 'Breakin'' (Score: 1.00)
✗ 'Snapshot Safari with Thunderhook' - No match found
✓ 'Esoterics Class w

In [ ]:
# Step 5: Generate user-to-role assignments
# Make sure you've reviewed the mappings file first!
assignments_df = generate_user_role_assignments(schedule_events_df, discord_events_df, mappings_df)

# IMPORTANT: Review outputs/user_role_assignments_for_review.csv and save your corrections
# as outputs/user_role_assignments_reviewed.csv before proceeding!

In [ ]:
# Step 6: Test on your test server first!
# This will do a DRY RUN to show what would happen
await apply_roles_to_users(assignments_df, TEST_SERVER_ID, dry_run=True)

In [ ]:
# Step 7: Apply to test server for real (optional)
# await apply_roles_to_users(assignments_df, TEST_SERVER_ID, dry_run=False)

In [ ]:
# Step 8: FINAL PRODUCTION RUN - BE VERY CAREFUL!
# This will apply all roles to the production server
# await apply_roles_to_users(assignments_df, PRODUCTION_SERVER_ID, dry_run=False)

In [ ]:
# Test database connection
test_conn = get_database_connection()
if test_conn:
    print("\n✓ Database connection successful!")
    test_conn.close()
else:
    print("\n✗ Could not connect to database")
    print("\nTroubleshooting steps:")
    print("1. Make sure your .env file is in: D:\\bench\\ScheduleBot\\role_migration_system\\")
    print("2. Verify your .env file contains:")
    print("   MYSQL_SERVER=your-rds-endpoint.region.rds.amazonaws.com")
    print("   MYSQL_USER=your-username")
    print("   MYSQL_USER_PW=your-password")
    print("   MYSQL_DB=ScheduleBot")
    print("3. Ensure RDS is set to publicly accessible")
    print("4. Check security group allows your IP on port 3306")